In [ ]:
%pip install fashion-clip
%pip install faiss-cpu

In [10]:
# Generar un pandas con el dataset inditextech_hackupc_challenge

import pandas as pd
import numpy as np

# Cargamos el dataset
df = pd.read_csv('inditextech_hackupc_challenge_images.csv')

batch_size = 10000
start = 31216
df_batch = df[start:start+batch_size]

In [ ]:
"""
import requests
import numpy as np
import pandas as pd
from PIL import Image
import io

# Suponiendo que tienes un DataFrame llamado df con una columna llamada 'IMAGE_VERSION_3' que contiene las URLs de las imágenes

# Función para leer la imagen desde la URL y convertirla en un array de numpy
def read_image(url):
    try:
        response = requests.get(url)
        image_bytes = io.BytesIO(response.content)
        image = Image.open(image_bytes)
    except:
        return None 
    return np.array(image)

# Generamos el pandas
df_batch['id'] = df_batch.index
# leer la imagen de la tercera url de la fila y convertirla a numpy array
df_batch['image'] = df_batch['IMAGE_VERSION_3'].apply(lambda x: read_image(x) if pd.notnull(x) else None)
df_batch
"""

In [ ]:
import os
import pandas as pd
import requests
from random import randint
import io
import time

# Suponiendo que df_batch ya está definido
inditex_images = pd.DataFrame(columns=['id', 'path_to_image'])

# Funcion para detectar que la respuesta es una imagenç
def is_image(response):
    image_formats = ("image/png", "image/jpeg", "image/jpg")
    return response.headers["content-type"] in image_formats

# Añadir los items al DataFrame inditex_images
for index, row in df_batch.iterrows():
    url = row['IMAGE_VERSION_3']
    if pd.notna(url):
        try:
            response = requests.get(url)
            response.raise_for_status() # Lanzar una excepción si la respuesta no es 200
            # Escribir la imagen en un archivo
            filename = f'inditex_images/{index}.jpg'
            with open(filename, 'wb') as file:
                file.write(response.content)
                
            # Agregar la ruta de la imagen al DataFrame
            new_row = pd.DataFrame({'id': [index], 'path_to_image': [filename]})
            inditex_images = pd.concat([inditex_images, new_row], ignore_index=True)
            
        except Exception as e:
            print(f"No se pudo descargar o guardar la imagen {index} debido a: {e}")
            time.sleep(randint(0, 10)/10)  # Esperar un segundo antes de intentar la siguiente descarga
    else:
        print(f'No hay URL válida para la imagen {index}')

# Mostrar el DataFrame actualizado
print(inditex_images)


In [11]:
from multiprocessing import Pool

# Funcion para detectar que la respuesta es una imagenç
def is_image(response):
    image_formats = ("image/png", "image/jpeg", "image/jpg")
    return response.headers["content-type"] in image_formats

def worker(url, df_batch_fragment):
    for index, row in df_batch_fragment.iterrows():
        url = row['IMAGE_VERSION_3']
        if pd.notna(url):
            try:
                response = requests.get(url)
                response.raise_for_status() # Lanzar una excepción si la respuesta no es 200
                # Escribir la imagen en un archivo
                filename = f'inditex_images/{index}.jpg'
                with open(filename, 'wb') as file:
                    file.write(response.content)
                
            except Exception as e:
                print(f"No se pudo descargar o guardar la imagen {index} debido a: {e}")
                time.sleep(randint(0, 10)/10)
        else:
            print(f'No hay URL válida para la imagen {index}')

# Dividir el DataFrame en fragmentos
num_processes = 10
df_batch_fragments = np.array_split(df_batch, num_processes)

# Crear un pool de procesos
with Pool(num_processes) as pool:
    pool.starmap(worker, [(url, df_batch_fragment) for url, df_batch_fragment in zip(df_batch_fragments, df_batch_fragments)])

/home/pablo/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


No hay URL válida para la imagen 22217
No hay URL válida para la imagen 24217
No hay URL válida para la imagen 30217
No hay URL válida para la imagen 27217
No hay URL válida para la imagen 28218
No hay URL válida para la imagen 28219
No hay URL válida para la imagen 30219
No hay URL válida para la imagen 24219
No hay URL válida para la imagen 24220
No hay URL válida para la imagen 29219
No hay URL válida para la imagen 27221
No hay URL válida para la imagen 24223
No hay URL válida para la imagen 23220
No hay URL válida para la imagen 30224
No hay URL válida para la imagen 21222
No hay URL válida para la imagen 26223
No hay URL válida para la imagen 27224
No hay URL válida para la imagen 24226
No hay URL válida para la imagen 25224
No hay URL válida para la imagen 30227
No hay URL válida para la imagen 30228
No hay URL válida para la imagen 21225
No hay URL válida para la imagen 24228
No hay URL válida para la imagen 24229
No hay URL válida para la imagen 29225
No hay URL válida para la

In [ ]:
Image.open(io.BytesIO(requests.get(df_batch['IMAGE_VERSION_3'][7]).content)).save('test.jpg')

# Embedings

___

In [ ]:
# Cargar el pandas desde un archivo pkl
df = pd.read_pickle('inditextech_hackupc_challenge_images.pkl')

In [ ]:
# Generar un wordembeddings con el dataset
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# Tenemos un pandas con una imagen pasada a numpy array en la columna 'image'

# Función para generar un wordembeddings con las imágenes
def generate_wordembeddings(df):
    # Cargar el modelo preentrenado de word2vec
    model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
    
    # Generar el wordembeddings
    wordembeddings = []
    for i in range(len(df)):
        image = df['image'][i]
        if image is not None:
            wordembedding = np.mean([model[word] for word in image.flatten() if word in model], axis=0)
        else:
            wordembedding = None
        wordembeddings.append(wordembedding)
    return wordembeddings